After all, the fundamental reason we build predictive machine learning models is typically to model what would happen under a particular set of circumstances which have either not yet occurred or have occurred but for which we do not know the actual result.

	Hyperparameter tuning?
	Should I do BOTH data_with_paymentcols and data_without_paymentcols?
	Look at feature importances:
        barchart for top x features or all features with coefficient > y. 
        How can I find the p-value for features? Is that only in regression?
	Do Ridge Regression and Lasso Regression.
		Barchart for top features
	Make DataFrame with top features for my model, Lasso Regression, and Ridge Regression (or model and Ridge)
	Make inferences: 
		Does race show up in these features? Where?
		How do I interpret differences in RATES distinct from differences in CHARGES? A city doctor should CHARGE more, but should his rate b higher?
        Or should a rural (lowpop) doctor have higher rates bc he might charge less?
        Or might city doctors be cheap Public Health providers /Flu shots, thus paid in full?

    Look at extreme mistakes:
    Providers marked low but actually v high. Providers marked v high but actually low: barcharts of types of providers, racial majority, regions, subregions, histogram of submitted charges OR medicare payments, histogram of payment/service, histogram of payment/person. Boxplots of percent black / white / hispanic / asian for false very-highs and for false-lows



# Importing Modules and Checking Directory 

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
pd.__version__

'1.1.5'

In [3]:
os.getcwd()


'/Users/Livi/Projects/Black_mental_health/models'

In [4]:

df.head()

/Users/Livi/opt/anaconda3/envs/springboard/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,entity_code,state,provider_type,number_of_hcpcs,number_of_services,total_beneficiaries,total_submitted_charges,avg_beneficiary_age,avg_hcc_risk_score,submitted_charges_per_person,...,total_pop,percent_black,percent_white,percent_native_amer,percent_hispanic,percent_other,percent_asian,percent_two_or_more_races,majority,payrate_level
0,I,MD,Internal Medicine,19,1218.0,445,692640.00,76,2.2948,1556.49,...,42583.0,0.109645,0.839772,0.001573,0.018834,0.001127,0.007327,0.021464,majority_white,high
1,I,MD,Internal Medicine,16,1293.0,546,265250.00,80,2.1736,485.81,...,26448.0,0.108553,0.612523,0.003932,0.106360,0.006579,0.123299,0.038755,majority_white,high
2,I,MD,Physician Assistant,4,58.0,12,2728.42,65,0.8597,227.37,...,21010.0,0.579819,0.303475,0.002237,0.042075,0.006140,0.045740,0.017753,majority_black,very low
3,I,MD,Ophthalmology,30,3417.0,791,648023.00,79,1.2159,819.25,...,42868.0,0.309042,0.448306,0.002659,0.135416,0.004689,0.067883,0.031539,no_majority,low
4,I,MD,Internal Medicine,22,262.0,72,35250.00,68,3.0926,489.58,...,17136.0,0.498833,0.337768,0.004260,0.039974,0.004785,0.079890,0.034022,no_majority,low


In [5]:
y = df.payrate_level
X = pd.get_dummies(df.drop(columns='payrate_level'))

In [6]:
X.shape, y.shape

((1072142, 188), (1072142,))

In [7]:
kbest = pd.read_csv('../data/interim/kbest_features40.csv')
ridge = pd.read_csv('../data/interim/ridge_features40.csv')

In [8]:
kbest.head()

,Unnamed: 0,0
0,0,provider_type_Mass Immunizer Roster Biller
1,1,entity_code_O
2,2,provider_type_Centralized Flu
3,3,provider_type_Optometry
4,4,provider_type_Certified Registered Nurse Anest...


In [9]:
ridge.head()

,Unnamed: 0,0
0,provider_type_Registered Dietitian or Nutritio...,0.090226
1,provider_type_Medicare Diabetes Preventive Pro...,0.077150
2,provider_type_Mass Immunizer Roster Biller,0.076272
3,provider_type_Centralized Flu,0.075123
4,provider_type_Public Health or Welfare Agency,0.074401


In [10]:
kbest = list(kbest['0'])
ridge = list(ridge['Unnamed: 0'])

In [30]:
ridge.remove('medicare_participation_indicator_Y')

Scaling my features with Method 1 from Preprocessing: 

Just continuous features not binary

In [14]:
from sklearn.preprocessing import PowerTransformer
import time

start= time.time()

scaler = PowerTransformer()

X_scaled=X.copy()

for col in X.columns:
    if col =='entity_code_I':
        break
    else:
        scaler.fit(np.array(X_scaled[col]).reshape(-1,1))
        i_scaled = scaler.transform(np.array(X_scaled[col]).reshape(-1,1))
        X_scaled[col] = i_scaled
        
        
end= time.time()

print('Time scaled:', (end-start)/60, 'minutes.')

Time scaled: 0.5407798528671265 minutes.


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled[kbest], y.values.ravel(), test_size = 0.2, random_state=2)

In [16]:
X_train.shape, y_train.shape

((857713, 40), (857713,))

In [17]:
import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score


In [18]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_jobs=-1)

start = time.time()

rf.fit(X_train, y_train)

end= time.time()
total = end - start

print("Time:", total/60, "minutes")

Time: 3.7817746003468833 minutes


In [23]:
start = time.time()

ypred_rf = rf.predict(X_test)

end = time.time()

f1_rf = f1_score(y_test, ypred_rf, average='weighted')
print('RF F1 Score:', f1_rf)
print('Time:',(end-start)/60, 'minutes')

RF F1 Score: 0.5020336864085725
Time: 0.3025496164957682 minutes


In [26]:
import time
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(n_jobs=-1)

start = time.time()

lr.fit(X_train, y_train)
ypred_lr = lr.predict(X_test)



f1_lr = f1_score(y_test, ypred_lr, average='weighted')

end= time.time()
total = end - start

print('LR F1 Score KBest:', f1_lr)
print("Time:", total/60, "minutes")

RF F1 Score: 0.5561281093622733
Time: 1.8151280641555787 minutes


#### LR with Ridge Features

In [32]:
import time
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X_scaled[ridge], y.values.ravel(), test_size = 0.2, random_state=2)

lr = LogisticRegression(n_jobs=-1)

start = time.time()

lr.fit(X_train, y_train)
ypred_lr_ridge = lr.predict(X_test)



f1_lr_ridge = f1_score(y_test, ypred_lr_ridge, average='weighted')

end= time.time()
total = end - start

print('LR F1 Score Ridge:', f1_lr_ridge)
print("Time:", total/60, "minutes")

LR F1 Score Ridge: 0.46062162431075276
Time: 1.7808371702829997 minutes


#### RF with Ridge Features

In [33]:
rf = RandomForestClassifier(n_jobs=-1)

start = time.time()

rf.fit(X_train, y_train)
ypred_rf_ridge = rf.predict(X_test)


f1_rf_ridge = f1_score(y_test, ypred_rf_ridge, average='weighted')
print('RF F1 Score:', f1_rf_ridge)


end= time.time()
total = end - start

print("Time:", total/60, "minutes")

RF F1 Score: 0.5136682698302621
Time: 4.399642511208852 minutes


In [34]:
scores = pd.DataFrame({'models':['RF, KBest', 'LR, KBest', 'LR Ridge', 'RF Ridge'],
             'F1 Scores': [f1_rf, f1_lr, f1_lr_ridge, f1_rf_ridge],
                      })
scores

,models,F1 Scores
0,"RF, KBest",0.502034
1,"LR, KBest",0.556128
2,LR Ridge,0.460622
3,RF Ridge,0.513668


#### BEST MODEL WITH FEATURE SELECTION: LR with KBEST FEATS

from sklearn.metrics import plot_confusion_matrix

start = time.time()

plt.figure(figsize=(20,20))
z = plot_confusion_matrix(rf, X_test, y_test, labels=['very low','low','medium','high','very high'])
#plt.savefig('confusion_matrix_RF.png')

end=time.time()


print('Time:', (end-start)/60, 'minutes')
z

# Tuning

In [40]:
from sklearn.metrics import make_scorer, f1_score

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid searchsco
grid = dict(solver=solvers,penalty=penalty,C=c_values)
scorer = make_scorer(f1_score, average='weighted')
grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=5, scoring=scorer)
grid_result = grid_search.fit(X, y)

KeyboardInterrupt: 

##### Using suggested metrics

In [69]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.2, random_state=2)


rf_tune1 = RandomForestClassifier(n_jobs=-1, max_depth=40, max_features=2, n_estimators=60)

start = time.time()

rf_tune1.fit(X_train[:100000], y_train[:100000])
ypred_rf_tune1 = rf_tune1.predict(X_test)
f1_rf_tune1 = f1_score(y_test, ypred_rf_tune1, average='weighted')


end= time.time()
total = end - start

print('RF F1 Score:', f1_rf_tune1)
print("Time:", total/60, "minutes")

Time: 0.43638649384180705 minutes


In [72]:
#plt.figure(figsize=(20,20))
#z = plot_confusion_matrix(rf_tune1, X_test, y_test, labels=['very low','low','medium','high','very high'])
#plt.savefig('confusion_matrix_RF.png')
#z

RF F1 Score: 0.6229051185250478
Time: 0.04270068804423014 minutes


<Figure size 1440x1440 with 0 Axes>

In [73]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.2, random_state=2)


rf_tune2 = RandomForestClassifier(n_jobs=-1, max_depth=40, max_features=20, n_estimators=80)

start = time.time()

rf_tune2.fit(X_train[:100000], y_train[:100000])
ypred_rf_tune2 = rf_tune2.predict(X_test)
f1_rf_tune2 = f1_score(y_test, ypred_rf_tune2, average='weighted')

end= time.time()
total = end - start

print('RF F1 Score:', f1_rf_tune2)
print("Time:", total/60, "minutes")

RF F1 Score: 0.6222025840609103
Time: 0.6454835653305053 minutes


In [74]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

start = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.2, random_state=2)
rf_tune3 = RandomForestClassifier(n_jobs=-1, max_depth=50, max_features=25, n_estimators=100)
rf_tune3.fit(X_train[:100000], y_train[:100000])
ypred_rf_tune3 = rf_tune3.predict(X_test)
f1_rf_tune3 = f1_score(y_test, ypred_rf_tune3, average='weighted')

end= time.time()
total = end - start

print('RF F1 Score:', f1_rf_tune3)
print("Time:", total/60, "minutes")

RF F1 Score: 0.6238192652933058
Time: 1.0376463810602823 minutes


In [75]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

start = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.2, random_state=2)
rf_tune4 = RandomForestClassifier(n_jobs=-1, max_depth=40, max_features=25, n_estimators=100)
rf_tune4.fit(X_train[:100000], y_train[:100000])
ypred_rf_tune4 = rf_tune4.predict(X_test)
f1_rf_tune4 = f1_score(y_test, ypred_rf_tune4, average='weighted')

end= time.time()
total = end - start

print('RF F1 Score:', f1_rf_tune4)
print("Time:", total/60, "minutes")

RF F1 Score: 0.6241083908163775
Time: 1.1664363781611125 minutes


##### Using Randomized Search CV

In [86]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

start = time.time()

params = {'max_depth':randint(1,50),
         'max_features':randint(1,40),
          'n_estimators':randint(10,100),
         'criterion':['gini']}

rf = RandomForestClassifier()

scorer = make_scorer(f1_score, average = 'weighted')

randomized_rf = RandomizedSearchCV(rf, params, cv=5, scoring=scorer)

randomized_rf.fit(X_train[:100000], y_train[:100000])


end=time.time()
print('time:', (end-start)/60, 'minutes')
print('best params:', randomized_rf.best_params_)
print('best score:', randomized_rf.best_score_)

time: 19.99403643210729 minutes
best params: {'criterion': 'gini', 'max_depth': 40, 'max_features': 35, 'n_estimators': 80}
best score: 0.6219702043294586


In [91]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

start = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.2, random_state=2)
rf_tune5 = RandomForestClassifier(n_jobs=-1, max_depth=40, max_features=35, n_estimators=80)
rf_tune5.fit(X_train[:100000], y_train[:100000])
ypred_rf_tune5 = rf_tune5.predict(X_test)
f1_rf_tune5 = f1_score(y_test, ypred_rf_tune5, average='weighted')

end= time.time()
total = end - start

print('RF F1 Score:', f1_rf_tune5)
print("Time:", total/60, "minutes")

RF F1 Score: 0.6235888024216302
Time: 1.06476598183314 minutes


Saving Model Metrics Table

In [193]:
import dataframe_image as dfi
dfi.export(scores,"model_metrics.png")

In [194]:
scores.to_csv('../reports/figures/model_scores.csv')

Saving Feature Importances RF

In [203]:
rf_imp = pd.DataFrame({'Features': X.columns, 
              'RF Feature Importances': [format(i, '8f') for i in rf.feature_importances_]}
            ).sort_values('RF Feature Importances', ascending=False).head(20)
rf_imp.to_csv('../references/rf_feature_imp.csv')

In [346]:
rf_imp.head(20)

,Features,RF Feature Importances
8,submitted_charges_per_service,0.166715
6,submitted_charges_per_person,0.082571
7,services_per_person,0.077495
3,total_submitted_charges,0.052601
0,number_of_hcpcs,0.043647
1,number_of_services,0.043515
5,avg_hcc_risk_score,0.039671
2,total_beneficiaries,0.032824
4,avg_beneficiary_age,0.028493
15,percent_asian,0.024522


Saving Errors from Random Forest Models as a CSV. I want to see which doctors are mislabeled and project why?

In [38]:
idx = X_test.index
examine_errors = pd.read_csv('../data/interim/merged_medicare_before_dropping.csv',index_col=0).iloc[idx]
examine_errors = pd.concat([examine_errors.reset_index(drop=True),
                                   pd.Series(ypred_rf).rename('ypred_rf'),
                                   pd.Series(y_test).rename('ytest')], axis=1)

/Users/Livi/opt/anaconda3/envs/springboard/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Here I'm saving the most extreme errors.

In [46]:
import dataframe_image as dfi

falselow = examine_errors[
    (examine_errors.ypred_rf == 'very low') & (examine_errors.ytest == 'very high')]

falsehigh = examine_errors[
    (examine_errors.ypred_rf == 'very high') & (examine_errors.ytest == 'very low')]

falselow = falselow[['entity_code','state','provider_type','number_of_services','number_of_hcpcs','total_beneficiaries','total_submitted_charges','avg_hcc_risk_score','submitted_charges_per_person','services_per_person','submitted_charges_per_service','region','subregion','total_pop','majority','ytest','ypred_rf']]
falsehigh = falsehigh[['entity_code','state','provider_type','number_of_services','number_of_hcpcs','total_beneficiaries','total_submitted_charges','avg_hcc_risk_score','submitted_charges_per_person','services_per_person','submitted_charges_per_service','region','subregion','total_pop','majority','ytest','ypred_rf']]




falselow.shape, falsehigh.shape

((60, 17), (109, 17))

In [47]:
falselow.head()

,entity_code,state,provider_type,number_of_services,number_of_hcpcs,total_beneficiaries,total_submitted_charges,avg_hcc_risk_score,submitted_charges_per_person,services_per_person,submitted_charges_per_service,region,subregion,total_pop,majority,ytest,ypred_rf
3730,I,CA,Family Practice,102.0,8,26,4210.00,1.9780,161.92,3.92,41.27,West,Pacific,71422.0,no_majority,very high,very low
5546,I,FL,Nurse Practitioner,28.0,10,14,2860.00,0.6400,204.29,2.00,102.14,South,South Atlantic,21807.0,majority_white,very high,very low
22823,I,MS,Family Practice,366.0,12,114,12720.34,1.0595,111.58,3.21,34.76,South,East South Central,30050.0,majority_white,very high,very low
23207,I,WA,Family Practice,31.0,7,25,1711.00,1.1905,68.44,1.24,55.19,West,Pacific,43285.0,majority_white,very high,very low
23339,I,DE,Certified Nurse Midwife,38.0,6,28,2683.00,0.6956,95.82,1.36,70.61,South,South Atlantic,25134.0,majority_white,very high,very low


In [48]:
falsehigh.head()

,entity_code,state,provider_type,number_of_services,number_of_hcpcs,total_beneficiaries,total_submitted_charges,avg_hcc_risk_score,submitted_charges_per_person,services_per_person,submitted_charges_per_service,region,subregion,total_pop,majority,ytest,ypred_rf
558,I,MS,Nurse Practitioner,151.0,22,65,6362.00,1.2124,97.88,2.32,42.13,South,East South Central,14874.0,majority_white,very low,very high
1296,I,CA,Pediatric Medicine,46.0,7,38,1461.82,1.2552,38.47,1.21,31.78,West,Pacific,65016.0,majority_hispanic,very low,very high
1427,I,OH,Registered Dietitian or Nutrition Professional,78.0,3,49,6682.00,1.2227,136.37,1.59,85.67,Midwest,East North Central,27011.0,majority_white,very low,very high
2789,I,KY,Nurse Practitioner,310.0,32,119,7781.04,1.5591,65.39,2.61,25.10,South,East South Central,6507.0,majority_white,very low,very high
3373,I,WA,Physician Assistant,125.0,14,76,4747.78,1.1872,62.47,1.64,37.98,West,Pacific,43078.0,majority_white,very low,very high



falselow.to_csv('../references/falselow.csv')
falsehigh.to_csv('../references//falsehigh.csv')